# Projeto Integrador FATEC Ourinhos - NovAmérica Agrícola Tarumã   * Pedro de Azevedo Martins

Dados drecebidos da empresa NovAmérica do setor de manutenção de oficina mecânica  Maio de 2021

# Utilizando ML para prever horas de manutenção por sub-sistema e melhor funcionario indicado para manutenção

# Importações de bibliotecas e dados

Importando as funções e bibliotecas que serão usadas

In [ ]:
from sklearn.ensemble import RandomForestRegressor # Modelo de regressão escolhido
from sklearn.metrics import mean_absolute_error # Métrica para validação e qualidade do modelo
from sklearn.model_selection import train_test_split
#from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score # Coeficiente de determinação ou R², outra métrica de qualidade
import pandas as pd
import datetime

In [ ]:
#dataFrame = pd.read_csv('FullData.csv')
dataFrame = pd.read_csv('drive/MyDrive/FullData.csv')
dataFrame.head()

,Classe de Manutenção - Código,Classe de Manutenção - Descrição,Data de Entrada,Data de Saída,Funcionário - Código,Funcionário - Descrição,Sistema Veicular - Código,Sistema Veicular - Código.1,Subsistema Veicular - Código,Subsistema Veicular - Descrição,\nHoras,\nSistema Veicular,\nFuncionário,\nData de Saída,\nData de Entrada,\nClasse de Manutenção,\nGeral
0,29,Corret. Emergencial,1/4/2020,1/4/2020,22955,Marcelo Venceslau da Silva,1400,SISTEMA HIDRÁULICO,1402,Cilindro hidráulico,2.35,100.0,100.0,2.10,0.11,0.0,0.0
1,29,Corret. Emergencial,1/4/2020,1/4/2020,30413,Agnaldo Aparecido Monteiro,1300,SISTEMA ELÉTRICO,1312,Indicadores Diversos,4.74,100.0,100.0,4.23,0.23,0.0,0.0
2,29,Corret. Emergencial,1/4/2020,1/4/2020,41598,Ademir de Jesus Aparecido,1400,SISTEMA HIDRÁULICO,1410,Mangueiras,2.99,100.0,100.0,2.67,0.15,0.0,0.0
3,29,Corret. Emergencial,1/4/2020,1/4/2020,50082,Alessandro Miranda,1000,PNEUS E AROS,1003,Pneus traseiros,0.29,100.0,100.0,0.26,0.01,0.0,0.0
4,29,Corret. Emergencial,1/4/2020,1/4/2020,52596,Josenaldo Silva Sousa,1300,SISTEMA ELÉTRICO,1301,Alternador,3.35,100.0,100.0,2.99,0.16,0.0,0.0


# Pré-processamento

Lidando com colunas irregulares

In [ ]:
# a seguir é modificado os rotulos que possuem os "/n" antes

dataFrame = dataFrame.rename(columns={'\nHoras':'Horas','\nSistema Veicular' : 'Sis. Veicular', '\nFuncionário' : 'Funcionario', '\nClasse de Manutenção' : 'Classe Manut.', '\nGeral' : 'Geral'}) #\nData de Saída' : 'Data de Saida', '\nData de Entrada' : 'Data de Entrada',
dataFrame.head()

,Classe de Manutenção - Código,Classe de Manutenção - Descrição,Data de Entrada,Data de Saída,Funcionário - Código,Funcionário - Descrição,Sistema Veicular - Código,Sistema Veicular - Código.1,Subsistema Veicular - Código,Subsistema Veicular - Descrição,Horas,Sis. Veicular,Funcionario,\nData de Saída,\nData de Entrada,Classe Manut.,Geral
0,29,Corret. Emergencial,1/4/2020,1/4/2020,22955,Marcelo Venceslau da Silva,1400,SISTEMA HIDRÁULICO,1402,Cilindro hidráulico,2.35,100.0,100.0,2.10,0.11,0.0,0.0
1,29,Corret. Emergencial,1/4/2020,1/4/2020,30413,Agnaldo Aparecido Monteiro,1300,SISTEMA ELÉTRICO,1312,Indicadores Diversos,4.74,100.0,100.0,4.23,0.23,0.0,0.0
2,29,Corret. Emergencial,1/4/2020,1/4/2020,41598,Ademir de Jesus Aparecido,1400,SISTEMA HIDRÁULICO,1410,Mangueiras,2.99,100.0,100.0,2.67,0.15,0.0,0.0
3,29,Corret. Emergencial,1/4/2020,1/4/2020,50082,Alessandro Miranda,1000,PNEUS E AROS,1003,Pneus traseiros,0.29,100.0,100.0,0.26,0.01,0.0,0.0
4,29,Corret. Emergencial,1/4/2020,1/4/2020,52596,Josenaldo Silva Sousa,1300,SISTEMA ELÉTRICO,1301,Alternador,3.35,100.0,100.0,2.99,0.16,0.0,0.0


Criando uma nova coluna, para representar o dia da semana (Seg, Ter, Qua...) usando a data de entradad

In [ ]:
dataFrame['Dia Semana'] = pd.to_datetime(dataFrame['Data de Entrada'],format= '%d/%m/%Y')
dataFrame['Dia Semana'] = dataFrame['Dia Semana'].dt.dayofweek
dataFrame.head()

,Classe de Manutenção - Código,Classe de Manutenção - Descrição,Data de Entrada,Data de Saída,Funcionário - Código,Funcionário - Descrição,Sistema Veicular - Código,Sistema Veicular - Código.1,Subsistema Veicular - Código,Subsistema Veicular - Descrição,Horas,Sis. Veicular,Funcionario,\nData de Saída,\nData de Entrada,Classe Manut.,Geral,Dia Semana
0,29,Corret. Emergencial,1/4/2020,1/4/2020,22955,Marcelo Venceslau da Silva,1400,SISTEMA HIDRÁULICO,1402,Cilindro hidráulico,2.35,100.0,100.0,2.10,0.11,0.0,0.0,2
1,29,Corret. Emergencial,1/4/2020,1/4/2020,30413,Agnaldo Aparecido Monteiro,1300,SISTEMA ELÉTRICO,1312,Indicadores Diversos,4.74,100.0,100.0,4.23,0.23,0.0,0.0,2
2,29,Corret. Emergencial,1/4/2020,1/4/2020,41598,Ademir de Jesus Aparecido,1400,SISTEMA HIDRÁULICO,1410,Mangueiras,2.99,100.0,100.0,2.67,0.15,0.0,0.0,2
3,29,Corret. Emergencial,1/4/2020,1/4/2020,50082,Alessandro Miranda,1000,PNEUS E AROS,1003,Pneus traseiros,0.29,100.0,100.0,0.26,0.01,0.0,0.0,2
4,29,Corret. Emergencial,1/4/2020,1/4/2020,52596,Josenaldo Silva Sousa,1300,SISTEMA ELÉTRICO,1301,Alternador,3.35,100.0,100.0,2.99,0.16,0.0,0.0,2


# Train test split

In [ ]:
# Y são os valores que queremos prever. no caso toda a coluna "Horas"
y = dataFrame['Horas']

#Os parametros são os valores que entrarão para as arvores de decisao *feature selection* (escolhas empíricas)
# X recebe as colunas com os parametros
parametros = ['Classe de Manutenção - Código', 'Funcionário - Código', 'Subsistema Veicular - Código', 'Dia Semana']
X = dataFrame[parametros]

#Dividindo o dataframe para usar parte em treinamento e parte em validadção
treinoX, valX, treinoy, valy = train_test_split(X, y, random_state=13)#random state gera uma semente de aleatoriedade sempre igual, para obter o mesmo resultado--
# em todas as vezes que o codigo é rodado

já temos os dados separado e prontos para serem adicionados ao modelo de ML, aqui usaremos o RandomForestRegressor

# Modelo de regressão

In [ ]:
modeloTreino = RandomForestRegressor(random_state=13)

modeloTreino.fit(treinoX, treinoy)

prediY = modeloTreino.predict(valX) # Atribuindo as prediçoes do modelo
print(prediY)

[3.04213333 2.03480667 1.60866667 ... 0.14390748 1.15835    1.72164167]


Vamos avaliar a capacidade de predição do nosso modelo usado o metodo MeanAbsoluteError comparando as predições geradas com o modelo e os valor reais separados anteriormente em valy (validação de y)

# Métrica

In [ ]:
maeModelo = mean_absolute_error(prediY, valy)
print(maeModelo) #Valores em horas
r2 = r2_score(prediY, valy) #coeficiente de determinaçao
print(r2)

1.5072897136509533
0.8421470944097318


Com o nosso modelo podemos prever o tempo total de uma manutenção com um erro médio aprox. de 1,5 Horas. No entando, agora usaremos todo o data frame para gerar um modelo com mais dados, afim de aumentar a acuracia da prediçoes, que será o modelo final onde as próximas inserções de dados terão seu tempo previsto.

# Modelo Final

In [ ]:
modeloFinal = RandomForestRegressor(random_state=13)
modeloFinal.fit(X, y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=13, verbose=0, warm_start=False)

O modelo final está pronto. podemos usar uma entrada de teste para prever o tempo de uma manunteção que venha ocorrer

Usaremos os campos seleceionado como parametros para prever. 
Nesse Exemplo um veículo teve uma correção emergencial (cód. 29),o funcionario Jefferson Gabriel Vilela (cód. 105184) foi designado para a tarefa, o problema fora no Sistema Hidráulico (Cód. 1400) no subsistema cilindro hidráulico (cód. 1402), que ocorreu em uma quarta-feira(2).

Nosso modelo tentará prever o tempo que o funcionario levara para realizar esse reparo

# Simulação

In [ ]:
dic = {
    'Classe de Manutenção - Código' : pd.Series(['29'], index = ['a']),
    'Funcionário - Código' : pd.Series(['105184'], index = ['a']), 
    'Dia Semana' : pd.Series(['2'], index = ['a']),
    'Subsistema Veicular - Código' : pd.Series(['1402'], index = ['a'])
}
x1 = pd.DataFrame(dic)
#x1 = [['Classe de Manutenção - Código','29'], ['Funcionário - Código','105184'], ['Sistema Veicular - Código','1400'], ['Subsistema Veicular - Código','1402']]
tempo = modeloFinal.predict(x1)
print(tempo)

[2.17867143]


# Função otimizadora para melhor funcionário

In [ ]:
def melhorFunc (clas, sem, sub):
  fun = open('drive/MyDrive/codFunc.csv','r')
  melhorT = 99999999
  MelhorF = ''
  for i in fun:
    dic = {
      'Classe de Manutenção - Código' : pd.Series([clas], index = ['a']),
      'Funcionário - Código' : pd.Series([str(i)], index = ['a']), 
      'Dia Semana' : pd.Series([sem], index = ['a']),
      'Subsistema Veicular - Código' : pd.Series([sub], index = ['a'])
    }
    x = pd.DataFrame(dic)
    tempo = modeloFinal.predict(x)
    if tempo < melhorT:
      melhorT = tempo
      MelhorF = i
  return melhorT, MelhorF

print(melhorFunc('29','2','1402'))

  

(array([0.48005238]), '101219\n')


saída de menor tempo de serviço prevista e código do funcionário